In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import mode
from collections import defaultdict

In [14]:
train_fname = 'data/train.csv'
test_fname = 'data/test.csv'
df = pd.read_csv(train_fname, sep=';')
df_test = pd.read_csv(test_fname, sep=';')

In [3]:
df = df.drop('VARIABLE_CIBLE', axis=1)

In [4]:
df = pd.concat((df, df_test)).reset_index()
df = df.drop('index', axis=1)

In [1]:
f_con = ['cited_age_min','cited_age_median','cited_age_max','cited_age_mean',
        'pct_NB_IPC','pct_NB_IPC_LY','oecd_NB_BACKWARD_PL','oecd_NB_BACKWARD_NPL',
        'IDX_ORIGIN','IDX_RADIC']
f_cat = ['APP_NB','APP_NB_PAYS','APP_NB_TYPE','VOIE_DEPOT','COUNTRY','SOURCE_BEGIN_MONTH','FISRT_APP_COUNTRY',
         'FISRT_APP_TYPE','LANGUAGE_OF_FILLING','FIRST_CLASSE','TECHNOLOGIE_SECTOR','TECHNOLOGIE_FIELD','MAIN_IPC',
         'FISRT_INV_COUNTRY','FISRT_INV_TYPE','SOURCE_CITED_AGE','SOURCE_IDX_ORI','SOURCE_IDX_RAD',
         'NB_CLASSES','NB_ROOT_CLASSES','NB_SECTORS','NB_FIELDS','INV_NB','INV_NB_PAYS','INV_NB_TYPE',
         'NB_BACKWARD_NPL','NB_BACKWARD_XY','NB_BACKWARD_I','NB_BACKWARD_AUTRE','NB_BACKWARD_PL',
         'NB_BACKWARD','oecd_NB_ROOT_CLASSES','PRIORITY_MONTH','FILING_MONTH','PUBLICATION_MONTH','BEGIN_MONTH','cited_n']

In [3]:
len(f_cat)

37

In [6]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [7]:
df_cat = MultiColumnLabelEncoder().fit_transform(df[f_cat].astype(str))

In [8]:
S = set(df_cat['FIRST_CLASSE'].value_counts()[0:2000].reset_index()[['index']].values[:,0])

In [9]:
Y = df_cat.values
for i in range(df_cat.shape[0]):
    if(not Y[i][9] in S):
        Y[i][9] = 41178
df_cat = pd.DataFrame(Y, columns=f_cat)

In [11]:
df_cat['APP_NB'].value_counts()

0     365121
8      19958
9       2465
10       630
16       590
11       178
12        95
13        54
14        20
15        14
4          6
1          5
2          4
6          2
5          2
7          1
3          1
Name: APP_NB, dtype: int64

In [12]:
X = pd.DataFrame(Imputer().fit_transform(df[f_con].values), columns=f_con)

In [13]:
df_2 = pd.concat((X, df_cat), axis=1)

In [26]:
n_samples = df.shape[0]
df_2[0:n_samples].to_csv('data/train_3.csv', sep=';')
df_2[n_samples:].to_csv('data/test_3.csv', sep=';')

In [49]:
encoder = OneHotEncoder()

In [50]:
n_samples = 259431

In [72]:
df1 = df_cat[0:n_samples]
df2 = df_cat[n_samples:]

In [66]:
df1.shape

(259431, 37)

In [67]:
df1 = encoder.fit_transform(df1)

In [78]:
df2['APP_NB_PAYS'].value_counts()

1     126899
3       2411
10       200
4        154
5         39
8          6
6          2
9          1
7          1
2          1
0          1
Name: APP_NB_PAYS, dtype: int64

In [77]:
df2 = encoder.transform(df2)

ValueError: unknown categorical feature present [  9 120] during transform.

In [75]:
df1.head(5)

,APP_NB,APP_NB_PAYS,APP_NB_TYPE,VOIE_DEPOT,COUNTRY,SOURCE_BEGIN_MONTH,FISRT_APP_COUNTRY,FISRT_APP_TYPE,LANGUAGE_OF_FILLING,FIRST_CLASSE,...,NB_BACKWARD_I,NB_BACKWARD_AUTRE,NB_BACKWARD_PL,NB_BACKWARD,oecd_NB_ROOT_CLASSES,PRIORITY_MONTH,FILING_MONTH,PUBLICATION_MONTH,BEGIN_MONTH,cited_n
0,0,1,1,0,86,0,127,1,6,4354,...,0,0,0,0,7,175,163,56,175,62
1,0,1,1,0,21,0,36,1,4,20022,...,0,0,0,0,12,221,233,108,221,62
2,0,1,1,1,48,0,64,1,6,41178,...,0,103,10,24,9,109,94,268,109,50
3,0,1,1,1,31,0,47,1,6,41178,...,0,0,54,118,25,90,96,246,90,62
4,0,1,1,1,21,0,36,1,4,41178,...,0,83,87,151,12,231,222,121,231,11


In [76]:
df2.head(5)

,APP_NB,APP_NB_PAYS,APP_NB_TYPE,VOIE_DEPOT,COUNTRY,SOURCE_BEGIN_MONTH,FISRT_APP_COUNTRY,FISRT_APP_TYPE,LANGUAGE_OF_FILLING,FIRST_CLASSE,...,NB_BACKWARD_I,NB_BACKWARD_AUTRE,NB_BACKWARD_PL,NB_BACKWARD,oecd_NB_ROOT_CLASSES,PRIORITY_MONTH,FILING_MONTH,PUBLICATION_MONTH,BEGIN_MONTH,cited_n
259431,0,1,1,1,94,1,24,1,9,2619,...,0,92,109,173,12,252,255,157,243,43
259432,0,1,1,0,86,0,127,1,6,21802,...,0,0,0,0,3,178,188,59,178,62
259433,0,1,1,1,48,0,64,1,6,41178,...,0,39,43,129,7,198,209,106,198,62
259434,0,1,1,0,86,0,127,1,6,22273,...,0,0,0,0,7,99,105,40,99,62
259435,0,1,1,0,4,0,8,1,6,22273,...,0,0,0,0,13,96,80,252,96,62
